In [4]:
import glob
from keras.models import Sequential
from keras.layers import MaxPooling2D,Flatten,Dropout,Conv2D,Dense
import numpy as np
import cv2
import matplotlib.pyplot as plt
#from sklearn.utils import shuffle

In [5]:
from keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen=ImageDataGenerator(rescale=1./255,shear_range=.2,zoom_range=.2)
test_gen=ImageDataGenerator(rescale=1./255)

In [8]:
train_data=train_gen.flow_from_directory(r"chest_xray/train",target_size=(250,250),class_mode='binary',batch_size=32)

Found 5216 images belonging to 2 classes.


In [9]:
test_data=test_gen.flow_from_directory(r"chest_xray/test",target_size=(250,250),class_mode='binary',batch_size=32)

Found 624 images belonging to 2 classes.


In [10]:
k_conv1=4
k_conv2=4
n_conv1=32
n_conv2=64

max_pool1=2
max_pool2=2

height=384
width=127
channels=3

hidden=64

In [18]:
model=Sequential()
model.add(Conv2D(n_conv1,(k_conv1,k_conv1),activation='relu',padding='same',input_shape=[250,250,channels]))
model.add(MaxPooling2D((5,5)))
model.add(Conv2D(n_conv2,(k_conv2,k_conv2),activation='relu',padding='same'))
model.add(MaxPooling2D((max_pool2,max_pool2)))
model.add(Flatten())
model.add(Dense(hidden,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model.fit_generator(train_data,validation_data=test_data,steps_per_epoch=30,epochs=5)

Epoch 1/5
30/30 [==============================] - 37s 1s/step - loss: 0.1711 - accuracy: 0.9240 - val_loss: 0.6870 - val_accuracy: 0.8189
Epoch 2/5
30/30 [==============================] - 28s 940ms/step - loss: 0.1741 - accuracy: 0.9229 - val_loss: 0.5614 - val_accuracy: 0.8718
Epoch 3/5
30/30 [==============================] - 30s 1s/step - loss: 0.1820 - accuracy: 0.9271 - val_loss: 0.4817 - val_accuracy: 0.8622
Epoch 4/5
30/30 [==============================] - 28s 935ms/step - loss: 0.2035 - accuracy: 0.9240 - val_loss: 0.6315 - val_accuracy: 0.8237
Epoch 5/5
30/30 [==============================] - 28s 939ms/step - loss: 0.1442 - accuracy: 0.9448 - val_loss: 0.5382 - val_accuracy: 0.8397


In [22]:
model.save('chestxray.h5')

In [22]:
import keras
model=keras.models.load_model('chestxray.h5')

In [26]:
import numpy as np
from keras.models import load_model
import cv2

def preprocessing(path):
    data=cv2.imread(path)
    data=cv2.resize(data,(250,250))
    data=np.array(data)
    data=data.reshape(-1,250,250,3)
    return data
    
data=preprocessing(r"chest_xray\val\NORMAL\NORMAL2-IM-1442-0001.jpeg")    
model=load_model('chestxray.h5')
predict=model.predict_classes(data)[0][0]

if predict==0:
    print("NORMAL")
else:
    print("PNEUMONIA DETECTED")

NORMAL
